In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import data_loader, models, train
from matplotlib import pyplot as plt
import numpy as np

In [3]:
hidden_dim = 400
latent_dim = 200
c = 1
batch_size = 64

lr = 1e-3
epochs = 10

In [39]:
dataloader_train, dataloader_valid = data_loader.CustomDataloaders.MNIST()
# device = torch.device('cuda:0')
device = torch.device('cpu')
model = models.VaeFc(hidden_dim=hidden_dim, latent_dim=latent_dim, c=c).to(device)